In [ ]:
import tensorflow as tf
print('Using Tensorflow '+tf.__version__)
assert tf.__version__>='1.0.0', ('You should use Tensorflow 1.0 or superior')

In [ ]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
#%matplotlib inline

In [ ]:
import sys
sys.path.append('../')
import os.path
import csv
import numpy as np

In [ ]:
from src.argparse import argparse
from src.region_to_bbox import region_to_bbox
from src.pprint_params import pprint_params
from src.Tracker import Tracker
from src.crops import *

In [ ]:
hp = {"z_lr":0}
evaluation = {"video": "vot2016_ball1"}
run = {"visualization":1}
hp,evaluation,run,env,design = argparse(hp, evaluation, run)
pprint_params((hp,evaluation,run,env,design),['tracker hyperparams','evaluation','run','environment','design'])

In [ ]:
video_folder = os.path.join(env.root_dataset, evaluation.dataset, evaluation.video)
gt_file = os.path.join(video_folder, 'groundtruth.txt')
gt = np.genfromtxt(gt_file, delimiter=',')
# bbox is in format <cx,cy,w,h>
init_bbox = region_to_bbox(gt[evaluation.start_frame])
frame_name_list = [f for f in os.listdir(video_folder) if f.endswith(".jpg")]
frame_name_list.sort()
assert len(gt) == len(frame_name_list), ('Number of frames and number of GT lines should be equal.')
num_frames = len(gt)
bboxes = np.zeros((num_frames,4)) # stores tracker's output for evaluation
# create an instance of the class Tracker and initialize it with groundtruth
tracker = Tracker(init_bbox, evaluation.start_frame, design, video_folder, frame_name_list)
# used to pad the crops
avg_chan = np.mean(tracker.frame, axis=(0,1))
# fixed multiplicative factors to scale up/down the target
scale_factors = hp.scale_step**np.linspace(-np.ceil(hp.scale_num/2), np.ceil(hp.scale_num/2), hp.scale_num)
scaled_exemplar = scale_factors * tracker.z_sz
min_z = hp.scale_min * tracker.z_sz
max_z = hp.scale_max * tracker.z_sz
min_x = hp.scale_min * tracker.x_sz
max_x = hp.scale_max * tracker.x_sz

In [ ]:
tracker.frame_padded, tracker.npad = pad_frame(tracker.frame, tracker.bbox.pos, tracker.z_sz, avg_chan);
# template of the target, will be fed to the graph
z_crop = extract_crops(tracker.frame_padded, tracker.npad, tracker.bbox.pos, tracker.z_sz, design.exemplar_sz)

In [ ]:
# # debug, visualize crop
# with tf.Session() as sess:
#     z = sess.run(z_crop)    
#     z = np.uint8(np.squeeze(z, axis=0))
#     plt.imshow(z)
#     plt.show()

In [ ]:
# cosine window to penalize large displacements
hann_1d = np.expand_dims(np.hanning(design.score_sz*hp.response_up), axis=0)
penalty = np.transpose(hann_1d) * hann_1d
penalty = penalty / np.sum(penalty)

In [ ]:
# initialize visualization
if run.visualization:    
    fig = plt.figure()
    ax = fig.gca()
    fig.show()
    ax.imshow(tracker.frame)
    fig.canvas.draw()    

In [ ]:
# > store initial bbox
for i in xrange(evaluation.start_frame+1, 2):
    print 'Frame '+str(i)+'/'+str(num_frames)
    # call tracker object to read new img
    tracker.get_frame(i, video_folder, frame_name_list)    
    scaled_search_area = tracker.x_sz * scale_factors
    tracker.frame_padded, tracker.npad = pad_frame(tracker.frame, tracker.bbox.pos, scaled_search_area[-1], avg_chan);
    x_crops = extract_crops(tracker.frame_padded, tracker.npad, tracker.bbox.pos, scaled_search_area, design.search_sz)
#     if run.visualization:
#         ax.imshow(tracker.frame)
#         fig.canvas.draw()

In [ ]:
# visualize crop
with tf.Session() as sess:
    x = sess.run(z_crop)    
    z = np.uint8(np.squeeze(z, axis=0))
    plt.imshow(z)
    plt.show()

In [ ]:
plt.close('all')

In [ ]:
# # initialize figure and visualize
# if run.visualization:    
#     fig, ax = plt.subplots(1)
#     r = patches.Rectangle(
#         (tracker.bbox.cx-tracker.bbox.w/2, tracker.bbox.cy-tracker.bbox.h/2),
#         tracker.bbox.w, tracker.bbox.h,
#         linewidth=2, edgecolor='r', fill=False)
#     ax.imshow(tracker.frame)
#     ax.add_patch(r)
#     plt.show()